In [1]:
import sys

sys.path.append('../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set

In [ ]:
_time_intervals = 30  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 12  # Number of execution scenarios                     (chi)

init = [1]

init = [1]
loc1_avail_list = init+[math.ceil(random.uniform(0,0.75)*100)/100.0 for _ in range (11)]

loc1_varying_factor = pandas.DataFrame(data={'com1_store20': init*3+[0]*(_exec_scenarios-3),
                                             'com1_discharge20': init*3+[0]*(_exec_scenarios-3),
                                             'com1_store50': [0]*_exec_scenarios,
                                             'com1_discharge50': [0]*_exec_scenarios})

loc4_varying_factor = pandas.DataFrame(data={'com1_store20': [0]*_exec_scenarios,
                                             'com1_discharge20': [0]*_exec_scenarios,
                                             'com1_store50': init*6+[0]*(_exec_scenarios-6),
                                             'com1_discharge50': init*6+[0]*(_exec_scenarios-6)})

loc7_varying_factor = pandas.DataFrame(data={'com1_store20': init*9+[0]*(_exec_scenarios-9),
                                             'com1_discharge20': init*9+[0]*(_exec_scenarios-9),
                                             'com1_store50': [0]*_exec_scenarios,
                                             'com1_discharge50': [0]*_exec_scenarios})

M = 1e6  # Big M

# Define temporal scales
scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])

demand_factor = [1] * _exec_scenarios